In [2]:
pip show tensorflow

Name: tensorflow
Version: 2.3.4
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/conda/lib/python3.7/site-packages
Requires: h5py, opt-einsum, numpy, protobuf, grpcio, termcolor, six, tensorboard, wheel, astunparse, gast, absl-py, google-pasta, wrapt, keras-preprocessing, tensorflow-estimator
Required-by: witwidget, tfx, tfx-bsl, tensorflow-transform, tensorflow-serving-api, tensorflow-model-analysis, tensorflow-io, tensorflow-data-validation, tensorflow-cloud, Keras, fairness-indicators, explainable-ai-sdk
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install tapas-table-parsing --user --no-deps -r requirements.txt
!pip install gspread --user
!pip install flask-cors --user
!pip install flask-ngrok --user
!pip install GPUtil
!pip install sentencepiece

Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment


In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import tapas
import gspread

In [6]:
! gsutil cp "gs://tapas_models/2020_08_05/tapas_wtq_wikisql_sqa_masklm_large_reset.zip" "tapas_model.zip" && unzip tapas_model.zip
! mv tapas_wtq_wikisql_sqa_masklm_large_reset tapas_model

Copying gs://tapas_models/2020_08_05/tapas_wtq_wikisql_sqa_masklm_large_reset.zip...
/ [1 files][  3.4 GiB/  3.4 GiB]   51.1 MiB/s                                   
Operation completed over 1 objects/3.4 GiB.                                      
Archive:  tapas_model.zip
   creating: tapas_wtq_wikisql_sqa_masklm_large_reset/
  inflating: tapas_wtq_wikisql_sqa_masklm_large_reset/bert_config.json  
  inflating: tapas_wtq_wikisql_sqa_masklm_large_reset/README.txt  
  inflating: tapas_wtq_wikisql_sqa_masklm_large_reset/model.ckpt.index  
  inflating: tapas_wtq_wikisql_sqa_masklm_large_reset/model.ckpt.data-00000-of-00001  
  inflating: tapas_wtq_wikisql_sqa_masklm_large_reset/vocab.txt  
  inflating: tapas_wtq_wikisql_sqa_masklm_large_reset/model.ckpt.meta  


In [7]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [8]:
gc = gspread.service_account(filename='felipe-sandbox-1308e1b345df.json')
sh = gc.open("efx_test_billing_sample")
print(sh.sheet1.get('A1'))

[['name']]


In [9]:
import gspread

googlesheets = {
    'TwnUserList':'efx_test_user_list',
    'Billing' : 'efx_test_billing_sample'
}

for sheetName in googlesheets:
  print (sheetName)
  print(googlesheets[sheetName])
  gsheets=gc.open(googlesheets[sheetName])
  exec(sheetName + "= gsheets.worksheet('Sheet1').get_all_values()")

TwnUserList
efx_test_user_list
Billing
efx_test_billing_sample


In [10]:
print (Billing)

[['name', 'department', 'price'], ['John Doe', 'finance', '$10.00'], ['Joe Smith', 'marketing', '$12.00'], ['Jessica Something', 'finance', '$5.00'], ['Jill Here', 'marketing', '$2.00'], ['Joanna There', 'finance', '$1.00'], ['John Doe', 'finance', '$5.00']]


In [11]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils


os.makedirs('results/wtq/tf_examples', exist_ok=True)
os.makedirs('results/wtq/model', exist_ok=True)
with open('results/wtq/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_model/model.ckpt{suffix}', f'results/wtq/model/model.ckpt-0{suffix}')

max_seq_length = 512
# max_seq_length=1024
vocab_file = "tapas_model/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def aggregation_to_string(index):
  if index == 0:
    return "NONE"
  if index == 1:
    return "SUM"
  if index == 2:
    return "AVERAGE"
  if index == 3:
    return "COUNT"
  raise ValueError(f"Unknown index: {index}")

def predict(table_data, queries):

  # Make string a list for function 
  queries=queries.split(",")

  #Function to run questions start
  # table = [list(map(lambda s: s.strip(), row.split("|"))) 
  #         for row in table_data.split("\n") if row.strip()]
  table=table_data
  # print(table)
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/wtq/tf_examples/test.tfrecord", examples)
  write_tf_example("results/wtq/tf_examples/random-split-1-dev.tfrecord", [])

  ! python -m tapas.run_task_main \
    --task="WTQ" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --reset_position_index_per_cell \
    --init_checkpoint="tapas_model/model.ckpt" \
    --bert_config_file="tapas_model/bert_config.json" \
    --mode="predict" 2> error

  results_path = "results/wtq/model/test.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  # htmltable=IPython.display.HTML(df.to_html(index=False))
  print()
  results=[]
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = sorted(prediction_utils.parse_coordinates(row["answer_coordinates"]))
      all_coordinates.append(coordinates)
      answers = '~+= '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      aggregation = aggregation_to_string(int(row["pred_aggr"]))
      # print(">", queries[position])
      print("Question: ", queries[position])
      answer_text = str(answers)
      if aggregation == 'SUM':
        split=answer_text.split("~+=")
        # print(split)
        arithmetic=sum(list(map(int,split)))
        # answer_text= f"{aggregation} = {arithmetic}"
        answer_text= f"Answer: {arithmetic}"
        # print(answer_text)
      elif aggregation == 'COUNT':
        split=answer_text.split("~+=")
        # print(split)
        arithmetic=len(split)
        # answer_text= f"{aggregation} = {arithmetic}"
        answer_text= f"Answer: {arithmetic}"
      elif aggregation == 'AVERAGE':
        split=answer_text.split("~+=")
        # print(split)
        arithmetic=sum(list(map(int,split)))/len(split)
        # answer_text= f"{aggregation} = {arithmetic}"
        answer_text= f"Answer: {arithmetic}"
        # print(answer_text)
      else:
        answer_text= f"Answer: {answer_text}"
        answer_text=answer_text.replace("~+=",",")
       
      print(answer_text)
      print('Coordinates Below:')
      print(all_coordinates)
      print("\n-----------------------------------\n")
      results.append(answer_text.replace("Answer: ",""))
  return queries,results

In [ ]:
from flask import Flask,request,jsonify
from flask import request

app = Flask(__name__)
app.config['JSON_SORT_KEYS'] = False

@app.route("/", methods=['GET'])
def home():
  return jsonify({'msg': 'Server running'})

@app.route("/question", methods=['GET'])
def question():
  #get args from get request
  args = request.args
  print(args)
  # getting argumnets
  questions = args.get('q')
  datasheet = args.get('ds')
 
  #if datasheet=='x':
  #  dataset = TwnSubmission
  if datasheet=='efx_test_user_list':
    dataset=TwnUserList
  #elif sheet=='x':
  #  dataset=DashboardQuestion
  #elif sheet=='x':
  #  dataset=Batch
  elif datasheet=='efx_test_billing_sample':
    dataset=Billing
  #elif sheet=='x':
  #  dataset=FulfillmentRate

  #submit to NQL function 
  queries,results=predict(dataset,questions)

  listReturn=[]
  for x in range(0,len(queries)):
    line={'index': x , 'question': queries[x] , 'answer': results[x]}
    listReturn.append(line)

  listReturn=jsonify(listReturn)
  return listReturn
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


ImmutableMultiDict([('q', '"can you get me a list of  managers?"'), ('ds', 'efx_test_user_list')])
is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


user,department,role
John Doe,finance,manager
Joe Smith,marketing,director
Jessica Something,finance,director
Jill Here,marketing,manager
Joanna There,finance,director


127.0.0.1 - - [07/Oct/2021 19:22:11] "GET /question?q="can%20you%20get%20me%20a%20list%20of%20%20managers?"&ds=efx_test_user_list HTTP/1.1" 200 -



Question:  "can you get me a list of  managers?"
Answer: 2
Coordinates Below:
[[(0, 0), (3, 0)]]

-----------------------------------

ImmutableMultiDict([('q', '"give me all the names?"'), ('ds', 'efx_test_billing_sample')])
is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


name,department,price
John Doe,finance,$10.00
Joe Smith,marketing,$12.00
Jessica Something,finance,$5.00
Jill Here,marketing,$2.00
Joanna There,finance,$1.00
John Doe,finance,$5.00


127.0.0.1 - - [07/Oct/2021 19:25:26] "GET /question?q="give%20me%20all%20the%20names?"&ds=efx_test_billing_sample HTTP/1.1" 200 -



Question:  "give me all the names?"
Answer: John Doe, Joe Smith, Jessica Something, Jill Here, Joanna There, John Doe
Coordinates Below:
[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0)]]

-----------------------------------

